<div class="alert alert-info">
<u><strong>Authors:</strong></u> <b>Alberto Vavassori</b> (alberto.vavassori@polimi.it), <b>Emanuele Capizzi</b> (emanuele.capizzi@mail.polimi.it) - 2024 - Politecnico di Milano, Italy <br>
Developed within the LCZ-ODC project, funded by the Italian Space Agency (agreement n. 2022-30-HH.0).
</div>

# Pan-sharpening of PRISMA imagery

### Introduction

This Notebook contains examples of **pan-sharpening methods for hyperspectral PRISMA imagery** ([Loncan et al. 2015<sup>1</sup>](#1)).

Pansharpening methods are implemented in the `methods.py` file. Functions are adapted to the PRISMA imagery context from a dedicated GitHub Repository[<sup>2</sup>](#2).

Note: a MATLAB Toolbox is also available for pansharpening of PRISMA data[<sup>3</sup>](#3).

#### PRISMA data specifications

Information about the spectral ranges of the hyperspectral and panchromatic data is available within the PRISMA product specification document[<sup>4</sup>](#4).

Specifically, spatial and spectral resolutions of the Hyperspectral (HS) - including Visible Near Infrared (VNIR) and Short Wave Infrared (SWIR) - and Panchromatic (PAN) bands are the following:

| Sensor | Cube | Spectral Range [nm] | Spatial Resolution [m] | #Bands |
| :---: | :---: | :----: | :---: | :---: |
| HS | VNIR | 400 - 1010 | 30 | 66 |
| HS | SWIR | 920 - 2500 | 30 | 171 |
| PAN | PAN | 400 - 700 | 5 | 1 |

### Resources
    
<span id="1">[<sup>1</sup>Loncan, L. et al. «Hyperspectral Pansharpening: A Review». *IEEE Geoscience and Remote Sensing Magazine* 2015, 3(3), 1879–1900](https://ieeexplore.ieee.org/document/7284770)</span><br>
<span id="2">[<sup>2</sup>GitHub Repo for multispectral imagery pansharpening](https://github.com/codegaj/py_pansharpening)</span><br>
<span id="3">[<sup>3</sup>Matlab Toolbox for PRISMA pansharpening](https://openremotesensing.net/knowledgebase/hyperspectral-and-multispectral-data-fusion/)</span><br>
<span id="4">[<sup>4</sup>PRISMA Product Specifications](http://prisma.asi.it/missionselect/docs/PRISMA%20Product%20Specifications_Is2_3.pdf)</span><br>
<span id="5">[<sup>5</sup>Aiazzi, B. et al. «Improving Component Substitution Pansharpening Through Multivariate Regression of MS+Pan Data». *IEEE Transactions on Geoscience and Remote Sensing* 2007, 45(10), 3230–3239](https://ieeexplore.ieee.org/document/4305344)</span><br>

### <a id='TOC_TOP'></a>Notebook content

</div>
    
 1. [Libraries and Data Preparation](#sec1)
 2. [Component Substitution Based Pansharpening](#sec2)

<hr>

<div class="alert alert-info" role="alert">
    
## <a id='sec1'></a>&#x27A4; 1. Libraries and Data Preparation

[Back to top](#TOC_TOP)
    
</div>

### Import useful libraries

<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
<a id='warning'></a> Make sure you have the following libraries installed in your working environment:
</div>

- `h5py`
- `sklearn`
- `ipyleaflet`
- `ipywidgets`
- `opencv` (i.e. `cv2`) -> install with `pip install opencv-python`
- `leafmap` -> install with `pip install -U leafmap`

Common libraries are also required (e.g. `numpy`, `rasterio`, `matplotlib`, `pandas`, `geopandas`, `shapely`).

In [ ]:
import numpy as np
import rasterio as rio
from rasterio.crs import CRS
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.merge import merge
from rasterio.plot import show_hist
from rasterio.warp import reproject, Resampling
import matplotlib.pyplot as plt
import matplotlib.image
import ipywidgets as widgets
from sklearn.decomposition import PCA
import geopandas as gpd
import h5py
import pandas as pd
import pyproj
import cv2
from shapely.geometry import Polygon
from rasterio import mask
from shapely.geometry import box
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, LayersControl, Rectangle
import leafmap

In [ ]:
# Import functions and set auto-reload
from methods import *
from metrics import *
from functions import *
%load_ext autoreload
%autoreload 2

### Date selection

Here it is possible to select the PRISMA image acquisition date, i.e. a specific image. Pansharpening will be performed only to the selected image.

In [ ]:
date_prisma_w = widgets.Dropdown(
    options = ['2023-02-09', '2023-03-22', '2023-04-08', '2023-06-17', '2023-07-10', '2023-08-08'],
    value = '2023-02-09',
    description = 'PRISMA date:',
    disabled = False,
    layout = {'width': 'max-content'},
    style = {'description_width': 'initial'}
)
date_prisma_w

In [ ]:
sel_prisma_date = date_prisma_w.value
print(f"The selected date is {sel_prisma_date}.")

According to the selected date, the folder containing the coregistered images (where the outputs of the Notebook will be saved as well) is the following:

In [ ]:
prisma_path = 'coregistered/' + sel_prisma_date.replace('-', '') + '/'
prisma_path

In [ ]:
prisma_meta = h5py.File('C:/Users/Utente/OneDrive - Politecnico di Milano/Progetti/Progetto LCZ-ODC/LCZ/PRISMA_S2_Processing/imagery_S2_PRISMA_raw/PRS_L2D_STD_20230209102412_20230209102416_0001.he5', 'r')

### Interactive selection of the area of interest (AOI)

In this section, it is possible to interactively select the area of interest.

<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
<a id='warning'></a> Pansharpening will only be carried out in the selected area. 

Running the pansharpening on the whole image extent would end up with a heavy file (up to 15 GB) [*extent: 30 km x 30 km; bands: ~240; spatial resolution: 5m*].
</div>

First, get the coordinates of the selected PRISMA image exent, by running the following function:

In [ ]:
df = get_prisma_extent(prisma_path, sel_prisma_date)

Draw an interactive map displaying the PRISMA image footprint.
<a id='MAP'></a>

In [ ]:
prisma_map, dc = draw_map(df['lat'].mean(), df['long'].mean(), 10)
area_extent = Rectangle(bounds=((list(df.iloc[3, :])), (list(df.iloc[0, :]))), color='red', fill_color='red', fill_opacity=0.1, name='PRISMA image extent')
prisma_map.add_layer(area_extent)
prisma_map

<div class="alert alert-success" role="alert">
<span>&#x2714;</span>
<a id='libraries'></a>
When the map appears, use the <strong>draw a marker</strong> option to select the central point of the AOI. Then just go on with the code.
</div>

In [ ]:
coords = dc.last_draw['geometry']['coordinates']

Select through the following widget the extent of the squared area (half of the square side).

*Note: there is a limit to 6 km, i.e. the square side cannot be larger than 12 km.*

In [ ]:
window_w = widgets.IntSlider(
    value=3000,
    min=0,
    max=12000,
    step=1,
    description='Window (m):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
window_w

In [ ]:
side = window_w.value

Call the function `draw_aoi` to extract the vertex coordinates of the AOI.

In [ ]:
lat1, long1, polygon = draw_aoi(coords, side)

In [ ]:
selected_area = Rectangle(bounds=((lat1.min(), long1.min()), (lat1.max(), long1.max())), color='blue', name='Selected AOI')
prisma_map.add_layer(selected_area)

<div class="alert alert-success" role="alert">
<span>&#x2714;</span>
<a id='libraries'></a>
The AOI is now displayed in the map.
</div>

[Go back to the map](#MAP)

Export the selection polygon to a Geopackage.

In [ ]:
polygon = gpd.read_file('./coregistered/' + sel_prisma_date.replace('-', '') + '/aoi_1.gpkg')

In [ ]:
polygon.to_file(prisma_path + "aoi_1.gpkg", driver="GPKG")

### [Optionally, select multiple AOIs]

<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
<a id='warning'></a> Optionally, you can iteratively perform pansharpening on subsets of the original image. <br>
Note: do not go closer to the borders! Pansharpening will be affected by border effects.
</div>

In [ ]:
# Open the last AOI
polygon = gpd.read_file(prisma_path + 'aoi_1.gpkg')

# Extract the geometry field
polygon_geometry = polygon.at[0, 'geometry']
coordinates_list = list(polygon_geometry.exterior.coords)

# Update the coordinates to define an adjacent AOI
# This will create an AOI horizontally (eastword) translated with respect to the previous one
# (this is defined based on the selected dimension of the window)

new_coordinates = [(x + (side * 2), y) for x, y in coordinates_list]
updated_polygon = Polygon(new_coordinates)
polygon.at[0, 'geometry'] = updated_polygon

# Save the new polygon to a geopackage
polygon.to_file(prisma_path + "aoi_2.gpkg", driver="GPKG")

### Import the HS and PAN bands and clip to the AOI

The co-registered bands of the HS (VNIR) and PAN cubes are imported and clipped to the selected AOI.

<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
<a id='warning'></a> Only the bands of the VNIR are used for pansharpening, since they cover the same region of the elecromagnetic spectrum as the PAN band. Accordingly, the SWIR bands will not be sharpened to the PAN resolution.
</div>

Set the paths of the panchromatic band `pan_full` and hyperspectral (VNIR) bands `hs_full` (coregistered).

In [ ]:
pan_full = prisma_path + 'PR_pan_' + sel_prisma_date.replace('-', '') + '_5m.tif'
hs_full = prisma_path + 'PR_VNIR_' + sel_prisma_date.replace('-', '') + '_30m.tif'

Set the path and name of the same images clipped to the selected AOI (`pan_path` and `hs_path`).

In [ ]:
pan_path = prisma_path + 'pan_1.tif'
hs_path = prisma_path + 'hs_1.tif'

Call the function `clip_pan_hs` defined in `methods.py` to clip the VNIR and PAN bands to the AOI and save them to GeoTIFF files.

In [ ]:
clip_pan_hs(hs_full, pan_full, polygon, hs_path, pan_path)

<div class="alert alert-success" role="alert">
<span>&#x2714;</span>
<a id='libraries'></a>
The VNIR and PAN bands have been clipped to the AOI and saved to GeoTIFF files. These will now be used as a starting point for pansharpening.
</div>

### Image preparation for pansharpening

Before applying the pansharpening methods, the VNIR bands have to be upsampled to the same resolution of the PAN band, and possibly all-zero value bands have to be removed.

Open the clipped images. The VNIR bands have to be upsampled to the same resolution of the PAN band to perform the pansharpening.

In [ ]:
with rio.open(pan_path) as src_pan:
    pan_data = src_pan.read()
    pan_data_meta = src_pan.meta

In [ ]:
with rio.open(hs_path) as src_hs:
    hs_data = src_hs.read()
    hs_data_meta = src_hs.meta

In [ ]:
band_threshold = 1e-8
hs_data = hs_data[~np.all(hs_data <= band_threshold, axis=(1,2))]

Extract the number of VNIR bands.

In [ ]:
n_bands = hs_data.shape[0]

In [ ]:
hs_data_meta['count'] = n_bands

In [ ]:
with rio.open(prisma_path + 'hs_VNIR_1.tif', 'w', **hs_data_meta) as dst:
    dst.write(hs_data)

Upsample the VNIR bands to the same resolution as PAN band. It is possible to select the sampling method.

Select the resampling method: this will be used to interpolate the VNIR bands to the same resolution as PAN.

In [ ]:
resampling_methods = { "nearest": Resampling.nearest,
                      "bilinear": Resampling.bilinear,
                      "cubic": Resampling.cubic,
                      "cubic_spline": Resampling.cubic_spline,
                      "lanczos": Resampling.lanczos,
                      "average": Resampling.average
}

resampling_methods_list = list(resampling_methods.keys())

resampling_w = widgets.Dropdown(
    options=resampling_methods_list,
    value='bilinear',
    description='Method:',
    disabled=False,
)
resampling_w

Set the metadata of the upsampled image, and specify the path where it will be saved.

In [ ]:
dst_meta = pan_data_meta.copy()
dst_meta.update({'count': n_bands})

In [ ]:
hs_5m_path = prisma_path + 'hs_5m_1.tif'

Use the function `resample_image` to upsample the VNIR bands and save them as a new GeoTIFF file.

In [ ]:
resampled_hs = resample_image(hs_data, hs_data_meta, dst_meta, hs_5m_path, resampling_methods[resampling_w.value])
print(f"Resampling using {resampling_w.value} done")

Finally, open the upsampled hyperspectral image at 5m resolution and remove the bands with all zeros.

In [ ]:
with rio.open(hs_5m_path) as src_hs_5m:
    hs_5m_data = src_hs_5m.read()
    hs_5m_data_meta = src_hs_5m.meta

In [ ]:
band_threshold = 1e-8
hs_5m_data = hs_5m_data[~np.all(hs_5m_data <= band_threshold, axis=(1,2))]

<div class="alert alert-info" role="alert">

## <a id='sec2'></a>&#x27A4; 2. Component Substitution Based Pansharpening

[Back to top](#TOC_TOP)

</div>

In this part of the Notebook, two Component Substitution (CS) based approaches are implemented for PRISMA image pansharpening, namely the **Principal Component Analysis (PCA)** and **Gram-Schmidt (GS)** methods. An adaptive version of the GS is also implemented, namely **Gram-Schmidt Adaptive (GSA)** ([Aiazzi et al. 2007<sup>5</sup>](#5)).

These methods rely upon the projection of the higher spectral resolution image into another space, in order to separate spatial and spectral information. The transformed data are sharpened by substituting the component that containes the spatial information with the PAN image. The greater the correlation between the PAN image and the replaced component, the less spectral distortion will be introduced by the fusion approach. The fusion process is completed by applying the inverse spectral transformation to obtain the fused image.

The general equation can be expressed as follows:

$\hat{H}_k = \tilde{H}_k + G_k (P-I_L) \quad k = 1, ..., N$

* $\hat{H}_k$ is the HS pansharpened image;
* $\tilde{H}_k$ is the HS image interpolated at PAN scale;
* $P$ is the PAN image;
* $G_k$ are the gain coefficients;
* $I_L$ is the so-called intensity component;

while $k$ denotes the k-th band ($N$ is the number of bands).

$G_k$ and $I_L$ are computed differently depending on the employed method.

$I_L$ can be expressed as $I_L = \sum_{i=1}^{N} w_i \tilde{H}_i$, but different formulations exist.

<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
<a id='warning'></a> Note about the input data shape for pansharpening functions:
</div>

Input data must be provided in the following order: `(n_bands, height, width)`, e.g. (63, 2136, 1860).

Nonetheless, in order to be saved as GeoTIFF in rasterio the order must be the following: `(height, width, n_bands)`, e.g. (2136, 1860, 63).

The metadata of the pansharpened image is the same as the panchromatic band, except for the band number. The following line sets the metadata of the pansharpened image.

In [ ]:
pansharpened_meta = pan_data_meta
pansharpened_meta.update({'count': hs_5m_data.shape[0]})

### **PCA**-based pansharpening

The hypothesis underlying the application of PCA to pansharpening is that the spatial information (shared by all the channels) is concentrated in the first PC, while the spectral information specific to each single band) is accounted for the other PCs.

The vectors $w_i$ and $G_k$ of coefficient vectors are derived by the PCA procedure applied to the HS image.

In [ ]:
# Launch the algorithm
PCA_pansharpened, variance_ratios, pc_comp = pan_pca(pan_data, hs_5m_data)

The higher the explained variance of the first Principal Component, the more reliable is the method.
It is hereafter possible to display the explained variance ratio of the first 10 Principal Components.

In [ ]:
x_bar_components = 4 #Change depending on the number of PC to be plotted
pc_names = ['PC' + str(i) for i in range(1, x_bar_components+1)]

In [ ]:
plt.plot(range(x_bar_components), variance_ratios[0:x_bar_components], marker = 'o')
# set the x-axis labels
plt.xticks(range(x_bar_components), pc_names)
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.show()

In [ ]:
# Save to file
with rio.open(prisma_path + 'pansharpened_PCA_1.tif', 'w', **dst_meta) as dst:
    dst.write(PCA_pansharpened)

### **GS**-based pansharpening

The fusion process starts by using, as the component, a synthetic low resolution PAN image $I_L$ at the same spatial resolution as the HS image. A complete orthogonal decomposition is then performed, starting with that component. The pansharpening procedure is completed by substituting that component with the PAN image, and inverting the decomposition.

Gain coefficients are computed as follows:

$G_k = \frac{cov(\tilde{H}_k, I_L)}{var(I_L)}$

while the weights:

$w_i = \frac{1}{N}$

meaning that the intensity component is computed as a mean of the HS bands.

In [ ]:
# Launch the algotithm
GS_pansharpened = pan_GS(pan_data, hs_5m_data)

In [ ]:
# Save to file
with rio.open(prisma_path + 'pansharpened_GS_1.tif', 'w', **dst_meta) as dst:
    dst.write(GS_pansharpened)

### **GSA**-based pansharpening

In adaptive version of the GS method, a linear regression between PAN and HS bands is performed. A synthetic intensity having a minimum mean-square error with respect to the reduced PAN, is computed. Accordingly, the weights $w_i$ are estimated by means of a *linear regression algorithm*.

In [ ]:
# Launch the algorithm
GSA_pansharpened = pan_GSA(pan_data, hs_data, hs_5m_data, 'local')

In [ ]:
# Save to file
with rio.open(prisma_path + 'pansharpened_GSA_1.tif', 'w', **dst_meta) as dst:
    dst.write(GSA_pansharpened)

### [If you have done piecewise pansharpening, merge the pansharpened images]

<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
<a id='warning'></a> Optionally, if you have computed more piecewise pansharpening, merge together the pansharpened images as well as the original hyperspectral images with the following code
</div>

In [ ]:
# merge the original PRISMA images
# define the path to the PRISMA images to be merged, and the name of the merged image
tiles = [prisma_path + 'hs_5m_1_nn.tif', prisma_path + 'hs_5m_2_nn.tif']
output_path = prisma_path + 'hs_5m_merged_nn.tif'

src_files_to_mosaic = []
for tile in tiles:
    src = rio.open(tile)
    src_files_to_mosaic.append(src)

mosaic, out_trans = merge(src_files_to_mosaic)
out_meta = src.meta.copy()

out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                 "crs": "epsg:32632"
                })

with rio.open(output_path, "w", **out_meta) as dest:
    dest.write(mosaic)

In [ ]:
# merge the pansharpened images (with GSA)
# define the path to the pansharpened images to be merged, and the name of the merged image
tiles = [prisma_path + 'pansharpened_GSA_1.tif', prisma_path + 'pansharpened_GSA_2.tif']
output_path = prisma_path + 'pansharpened_GSA_merged.tif'

src_files_to_mosaic = []
for tile in tiles:
    src = rio.open(tile)
    src_files_to_mosaic.append(src)

mosaic, out_trans = merge(src_files_to_mosaic)
out_meta = src.meta.copy()

out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                 "crs": "epsg:32632"
                })

with rio.open(output_path, "w", **out_meta) as dest:
    dest.write(mosaic)